In [1]:
import numpy as np
import pandas as pd

In [2]:
jokes_df = pd.read_csv('jokes.csv')
# jokes = jokes_df.Jokes.values
jokes = jokes_df.Jokes.values[:9000]

In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer

max_words = 50000
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(jokes)
sequences = tokenizer.texts_to_sequences(jokes)

In [4]:
vocab_size = len(tokenizer.word_index)

In [5]:
sentence_len = 6
pred_len = 1
train_len = sentence_len - pred_len
seq = []

for joke in sequences:
    if len(joke) >= sentence_len:
        for i in range(len(joke) - sentence_len):
            seq.append(joke[i: i + sentence_len])

reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))

x_train = np.array([i[:train_len] for i in seq])
y_train = np.array([i[-1] for i in seq])

In [6]:
x_train.shape, y_train.shape, vocab_size

((138151, 5), (138151,), 40173)

In [7]:
from scipy import sparse as sp

dummies = pd.get_dummies(y_train, columns=[str(i) for i in range(1, vocab_size + 1)])
# dummies = pd.get_dummies(y_train, columns=[str(i) for i in range(1, vocab_size + 1)], sparse=True)

new_dummies = {}
for i in range(1, vocab_size + 1):
    if i in dummies.columns:
        new_dummies.update({i: dummies[i]})
    else:
        new_dummies.update({i: np.zeros(y_train.shape[0])})
        # new_dummies.update({i: sp.csr_matrix(np.zeros(y_train.shape[0]))})
new_dummies = pd.DataFrame(new_dummies)

In [8]:
new_dummies.shape

(138151, 40173)

In [9]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

model = Sequential([
    Embedding(vocab_size + 1, 50, input_length=train_len),
    LSTM(100, return_sequences=True),
    LSTM(100),
    Dense(100, activation='relu'),
    Dropout(0.1),
    Dense(vocab_size, activation='softmax')
])

In [10]:
from tensorflow.keras.callbacks import ModelCheckpoint

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
filepath = "./models/model_weights_2.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [11]:
history = model.fit(x_train, new_dummies, epochs=1,
                    batch_size=128, callbacks=callbacks_list, verbose=1)

MemoryError: Unable to allocate 41.4 GiB for an array with shape (40173, 138151) and data type float64

In [32]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

def generate(model, seq, max_len=20):
    tokenized_sent = tokenizer.texts_to_sequences([seq])
    max_len = max_len + len(tokenized_sent[0])

    while len(tokenized_sent[0]) < max_len:
        padded_sentence = pad_sequences(tokenized_sent[-5:], maxlen=5)
        op = model.predict(np.asarray(padded_sentence).reshape(1,-1))
        tokenized_sent[0].append(op.argmax() + 1)

    return seq + ' ' + " ".join(map(lambda x : reverse_word_map[x],tokenized_sent[0]))

In [33]:
generate(model, 'Ехали в поезде')

'Ехали в поезде в поезде инструктирует скорочтения нашли недорого пойдешь тяжело непонятно я меня о себе могут уже подтянулось я кто не кто точно она'

In [29]:
generate(model, 'Приятель из Питера Сергей Дединский')

'Приятель из Питера Сергей Дединский приятель из питера сергей нового читать в природной шнура они кажется в конце могут бабушка совсем не понял что делают нам открыто российские цирков'